In [212]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import re
import pandas as pd

# 더 퍼스트 슬램덩크 네티즌 평점 리스트
url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=223800&target=after"

# requests.get
resp = requests.get(url)
print(resp)


# HTTP 요청이 성공했음을 나타내는 서버측 성공 응답 상태 코드 = Response 200

<Response [200]>


In [149]:
# 서버 응답 중에서 소스코드 부분을 출력
resp.text[150:500]

'\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\r\n\t\r\n\t\r\n\t\t\r\n\t\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n\r\n\r\n\r\n\r\n\r\n\r\n<meta property="me2:image" content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg"/>\r\n<meta property="me2:post_tag"'

In [150]:
# BeautifulSoup 함수로, HTML 문서 구조를 파싱 
soup = BeautifulSoup(resp.text, 'html.parser')

# 파싱한 내용을 담고 있는 객체의 자료형 확인
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [151]:
# title 태그 이름을 활용하여 영화 제목이 포함되어 있는 요소를 찾습니다.
title_tag = soup.find(name='title')
print(title_tag)

# 텍스트 부분만 추출합니다.
title_text = title_tag.get_text()
print(title_text)

<title>네티즌 평점 : 네이버 영화</title>
네티즌 평점 : 네이버 영화


In [152]:
# span 태그의 class 속성값을 활용하여 리뷰 갯수가 포함되어 있는 요소를 찾습니다.
count_tag = soup.find(name='div', attrs={'class':'h5_right_txt'})
print("div 태그: ", count_tag)

# count_tag 요소에서 em 태그 부분을 찾습니다. 
count_tag = count_tag.find(name='strong')
print("strong 태그: ", count_tag)

# 텍스트 부분만 추출합니다.
count_text = count_tag.get_text()
print("텍스트: ", count_text)

div 태그:  <div class="h5_right_txt">총 <strong class="c_88 fs_11">13026</strong>개의 평점이 있습니다.</div>
strong 태그:  <strong class="c_88 fs_11">13026</strong>
텍스트:  13026


In [153]:
review_list_tag = soup.find(name='tbody')

# review_list_tag 요소에 포함된 tr 태그를 모두 찾습니다. 
review_list_tags = review_list_tag.find_all(name='tr')
print("tr 태그의 수: ", len(review_list_tags))
print("\n")

# 첫 번째 원소를 출력합니다. 
print(review_list_tags[0])

tr 태그의 수:  10


<tr>
<td class="ac num">18732834</td>
<td class="title">
<a class="movie color_b" href="/movie/bi/mi/basic.naver?code=223800">더 퍼스트 슬램덩크</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
</div>
<br/>눈물은 안나는데 뭉클하고 가슴뛰더라 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('wwoe****', '2SXAp85kW7IxTnlO5bMXEXL2sY/UYW8t9CfmGOGxxFg=', '눈물은 안나는데 뭉클하고 가슴뛰더라', '18732834', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>
<td class="num"><a class="author" href="javascript:find_list('nickname','18732834');">wwoe****</a><br/>23.02.06</td>
</tr>


In [202]:
# 첫 번째 아이템에서 리뷰 제목(rli.title), 사용자(rli.uid), 내용(rli.content) 데이터를 추출합니다.

review_title = review_list_tags[0].find_all('a')[0].get_text()
print("제목:", review_title, "\n")

review_content = review_list_tags[0].find("a",{"class":"report"}).get("onclick").split("', '")[2]  # find_all(x) find(o)
print(r"내용:", review_content, "\n")

review_uid = review_list_tags[0].find_all('a')[2].get_text()
print(r"사용자:", review_uid, "\n")

review_score = review_list_tags[0].find('em').get_text()
print(r"평점:", review_score, "\n")

review_time = review_list_tags[0].find_all("td",{"class":"num"})[1].get_text()
review_time = review_time[-8:]
print(r"작성일자:", review_time, "\n")

제목: 더 퍼스트 슬램덩크 

내용: 눈물은 안나는데 뭉클하고 가슴뛰더라 

사용자: wwoe**** 

평점: 10 

작성일자: 23.02.06 



In [203]:
# 각 리뷰 페이지로 연결할 nid 값을 포함하는 Javascript 코드를 추출합니다.

review_nid = review_list_tags[0].find("a",{"class":"report"}).get("onclick").split("', '")[3]  # find_all(x) find(o)

review_nid

'18732834'

In [214]:
need_reviews_cnt = 1000
reviews = []
review_data=[]

#page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈 
for page in range(1,100):
    url = f'https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=223800&target=after&page={page}'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    reviews = soup.find_all("td",{"class":"title"})
    
    #한 페이지의 리뷰 리스트의 리뷰를 하나씩 보면서 데이터 추출
    for review in reviews:
        sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2]
        #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
        if sentence != "":
            movie = review.find("a",{"class":"movie color_b"}).get_text()
            score = review.find("em").get_text()
            review_data.append([movie,sentence,int(score)])
            need_reviews_cnt-= 1
    #현재까지 수집된 리뷰가 목표 수집 리뷰보다 많아진 경우 크롤링 중지        
    if need_reviews_cnt < 0:                                         
        break

In [217]:
dict_data = {
    'title' : movie,
    'score' : score,
    'review' : review_data  
}

df_data = pd.DataFrame(dict_data)

# 변환 결과를 확인
df_data

,title,score,review
0,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 보면서 계속 심장뛰는 영화, 10]"
1,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 미친 영화….. 너무 감동 ㅎㅎ, 10]"
2,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 관객들이 몰입할 준비를 하고 왔다는게 너무 놀랐어요. 솔솔솔..."
3,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 그냥 미쳣음 피가 끓는영화, 10]"
4,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 작화, OST 모두 뛰어남, 10]"
...,...,...,...
917,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 저는 일본스타일보다는 한국스타일 슬램덩크 만화를 더 좋아해서..."
918,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 이번 더 퍼스트 슬램덩크는 송태섭의 스토리로 잘 시작하는 것..."
919,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 우리가 원하는 &amp;#39;실사&amp;#39;의 방향,..."
920,더 퍼스트 슬램덩크,10,"[더 퍼스트 슬램덩크, 더퍼슬 때문에 만화책을 구입해서 다시 읽어보았다.고등학생시절..."


In [218]:
df_data.to_csv("naver_review.csv")